In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import os
import torch
import re
import string
import json

import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import torch.nn as nn
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW


In [ ]:
df=pd.read_csv('/kaggle/input/newdata/cleandata.csv')

In [ ]:
df.info()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df['text_length'] = df['Text'].apply(lambda x: len(x.split()))

# Plot distribution of text lengths
plt.figure(figsize=(10, 6))
sns.histplot(df['text_length'], bins=50, kde=True)
plt.title('Distribution of Text Lengths')
plt.xlabel('Text Length (Number of Words)')
plt.ylabel('Frequency')
plt.show()

# Display basic statistics of text lengths
text_length_stats = df['text_length'].describe()
text_length_stats

In [ ]:
# Calculate the frequency of each class
class_frequencies = df.iloc[:, 1:-1].sum().sort_values(ascending=False)

# Plot class frequencies
plt.figure(figsize=(12, 8))
class_frequencies.plot(kind='bar')
plt.title('Frequency of Each Class')
plt.xlabel('Class')
plt.ylabel('Frequency')
plt.xticks(rotation=90)
plt.show()

class_frequencies

In [ ]:
# Calculate the correlation matrix for the label columns
label_correlation = df.iloc[:, 2:-1].corr()

# Plot the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(label_correlation, cmap='coolwarm', linewidths=.1)
plt.title('Correlation Matrix of Labels')
plt.show()

In [ ]:
df=df.drop(['text_length'],axis=1)

In [ ]:
target_cols = [col for col in df.columns if col not in ['Text']]
len(target_cols)

In [ ]:
df

In [ ]:
target_dict = {index: column for index, column in enumerate(target_cols)}

# Display the resulting dictionary
print(target_dict)

In [ ]:
df.info()

In [ ]:
class_frequencies = {
    'cs.LG': 12905,
    'cs.AI': 8198,
    'cs.CV': 7856,
    'cs.CL': 6818,
    'stat.ML': 5106,
    'stat.TH': 3567,
    'math.ST': 3567,
    'eess.AS': 3199,
    'cs.SD': 3087,
    'cs.IT': 2967,
    'math.IT': 2967,
    'math.PR': 2893,
    'math.AP': 2769,
    'math.CO': 2756,
    'cs.CR': 2630,
    'cs.RO': 2080,
    'eess.SP': 1996,
    'math.NT': 1690,
    'math.QA': 1657,
    'eess.IV': 1633,
    'stat.ME': 1614,
    'cs.NI': 1584,
    'cs.DC': 1489,
    'cs.IR': 1368,
    'cs.LO': 1345,
    'cs.SE': 1225,
    'math.GR': 991,
    'cs.PL': 984,
    'math.AC': 858,
    'stat.AP': 858,
    'math.LO': 832,
    'cs.DB': 792,
    'econ.EM': 635,
    'math.CV': 600,
    'cs.GT': 595,
    'cs.DM': 590,
    'stat.CO': 575,
    'math.AT': 569,
    'q-fin.EC': 548,
    'econ.GN': 539,
    'q-fin.PR': 524,
    'q-fin.CP': 501,
    'q-fin.MF': 485,
    'q-fin.RM': 475,
    'cs.AR': 465,
    'q-bio.BM': 394,
    'q-fin.PM': 332,
    'cs.CE': 312,
    'q-bio.MN': 281,
    'q-bio.NC': 273,
    'econ.TH': 272,
    'q-fin.GN': 272,
    'q-bio.GN': 257,
    'q-fin.TR': 241,
    'q-bio.CB': 191,
    'q-bio.TO': 177,
    'cs.OS': 134
}


In [ ]:
frequency_threshold = 600
minority_classes = [class_name for class_name in target_cols if class_frequencies[class_name] < frequency_threshold]

In [ ]:
minority_classes

In [ ]:
total_samples = sum(class_frequencies.values())

# Number of classes
num_classes = len(class_frequencies)

# Calculate class weights
class_weights = {class_label: (total_samples / (num_classes * freq)) for class_label, freq in class_frequencies.items()}

# Normalize the weights to make them sum up to 1 (optional, but can be useful)
weight_sum = sum(class_weights.values())
normalized_class_weights = {k: v / weight_sum for k, v in class_weights.items()}

print("Class Weights:", class_weights)
print("Normalized Class Weights:", normalized_class_weights)

In [ ]:
import torch.nn.functional as F
# # class_weights = {class_name: 1.0 / np.log(frequency + 1) for class_name, frequency in class_frequencies.items()}
# class_weights = {class_name: 1.0 / frequency for class_name, frequency in class_frequencies.items()}

In [ ]:
weights_tensor = torch.tensor([normalized_class_weights[class_name] for class_name in target_cols], dtype=torch.float)

# If using a GPU, move the tensor to the same device as your model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights_tensor = weights_tensor.to(device)

print(weights_tensor)

In [ ]:
# class_weights 

In [ ]:
# weights_tensor = torch.tensor([class_weights[class_name] for class_name in target_cols], dtype=torch.float)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
weights_tensor = weights_tensor.to(device)

In [ ]:
weights_tensor

In [ ]:
# def custom_loss(outputs, targets, weights_tensor):
#     bce_loss = F.binary_cross_entropy_with_logits(outputs, targets, reduction='none')
#     weighted_loss = bce_loss * weights_tensor.expand_as(bce_loss)
#     return weighted_loss.mean()
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
# import torch
# import pandas as pd
# from torch.utils.data import Dataset, DataLoader
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
# from sklearn.metrics import accuracy_score, f1_score
# from sklearn.model_selection import train_test_split
# import os
# import torch
# import re
# import string
# import json

# import emoji
# import numpy as np
# import pandas as pd
# from sklearn import metrics
# from bs4 import BeautifulSoup
# import torch.nn as nn
# import transformers
# import torch
# from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
# from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW


In [ ]:
from sklearn.model_selection import train_test_split
df_train,df_test = train_test_split(df, test_size=0.13, random_state=42)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
print(df_train.shape,df_test.shape)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 6e-5
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
train_dataset = BERTDataset(df_train, tokenizer, MAX_LEN)
test_dataset = BERTDataset(df_test, tokenizer, MAX_LEN)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=4, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=4, shuffle=False, pin_memory=True)

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoModel


In [ ]:
class MultiLabelSciBERT(torch.nn.Module):
    def __init__(self, num_labels):
        super(MultiLabelSciBERT, self).__init__()
        self.sci_bert = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')
        self.classifier = torch.nn.Linear(768, num_labels)

    def forward(self, ids,mask):
        outputs = self.sci_bert(ids, attention_mask=mask)
        sequence_output = outputs[0][:, 0, :]  
        return self.classifier(sequence_output)
    
    
    


In [ ]:
 model = MultiLabelSciBERT(57)
 model.to(device);

In [ ]:
 model

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)



In [ ]:
import torch
from torch.cuda.amp import autocast
import torch
from torch.cuda.amp import GradScaler

# Define the GradScaler object
scaler = GradScaler()
def train(epoch):
    model.train()
    for _, data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype=torch.long)
        mask = data['mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)

        optimizer.zero_grad()

        with autocast():
            outputs = model(ids, mask)
            loss = loss_fn(outputs, targets )

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        if _ % 500 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')


In [ ]:
for epoch in range(EPOCHS):
    train(epoch) 

In [ ]:
model

In [ ]:
torch.save(model.state_dict(), 'model5.pth')


In [ ]:
def validation():
    model.eval() 
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(test_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask)
#              outputs = model(ids, mask)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation()
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
import pandas as pd
# Load the test dataset
test_df = pd.read_csv("/kaggle/input/kriti-24/test.csv")

# Preprocess the text data
test_df['Text'] = test_df['Title'] + ' ' + test_df['Abstract']
test_df = test_df.drop(['Title', 'Abstract'], axis=1)
test_df['Text']=test_df['Text'].str.lower()


In [ ]:
def remove_punct(text):
    punctuation =string.punctuation
    return text.translate(str.maketrans('' , '',punctuation))
test_df['Text']=test_df['Text'].apply(lambda x: remove_punct(x))

In [ ]:
from nltk.corpus import stopwords 
",".join(stopwords.words('english'))
STOPWRDS=set(stopwords.words('english'))

In [ ]:
def remove_stp(Text):
    return " ".join([word for word in Text.split() if word not in STOPWRDS])

In [ ]:
test_df['Text']=test_df['Text'].apply(lambda x: remove_stp(x))

In [ ]:
not_needed = [
     "using", "results", "performance", "show", "method", "approach",
    "also", "based", "problem", "proposed", "two", "information", "new", "large",
    "different", "study", "however",  "analysis", "one", "work", "used", "first",
    "set", "use", "existing", "present", "process", "demonstrate", "task", "general",
    "several", "due", "compared", "via", "moreover", "eg", "thus", "possible", "make",
    "like", "important", "key", "furthermore", "give", "state", "certain", "ie", "form",
    "allows", "finally", "often", "even", "many", "various", "well", "also", "however",
    "several", "due", "across", "may", "without", "among", "including", "particular",
    "especially", "either", "often", "even", "moreover", "thus", "ie", "eg", "although",
    "despite","would" ,"within" ,"",
]

import pandas as pd
def not_need(Text):
    return " ".join([word for word in Text.split() if word not in not_needed])

test_df['Text'] = test_df['Text'].apply(lambda x:not_need(x))

In [ ]:
class BERTTestDatasets(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

# Create the test dataset
result_dataset = BERTTestDatasets(test_df, tokenizer, MAX_LEN)


In [ ]:
result_loader = DataLoader(result_dataset, batch_size=VALID_BATCH_SIZE, shuffle=False, num_workers=4)


In [ ]:
def predict():
    model.eval()
    predictions = []
    with torch.no_grad():
        for _, data in enumerate(result_loader, 0):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model(ids, mask)
            predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return predictions

# Get predictions
test_predictions = predict()


In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to binary (True/False)
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols)
submission_df.insert(0, 'Id', test_df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
target_cols2 = [col for col in df.columns if col not in ['Text','Title','Abstract']]
len(target_cols2)

In [ ]:
import numpy as np
import pandas as pd

# Convert predictions to binary (True/False)
binary_predictions = np.array(test_predictions) >= 0.5

# Convert boolean values to integers (1/0)
binary_predictions_int = binary_predictions.astype(int)

# Create a DataFrame for submission
submission_df = pd.DataFrame(binary_predictions_int, columns=target_cols2)
submission_df.insert(0, 'Id', test_df.Id)
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [ ]:
template_df = pd.read_csv('/kaggle/input/kriti-24/sample_submission.csv')
required_column_order = template_df.columns.tolist()


In [ ]:
# Ensure that 'Id' is the first column
required_column_order.remove('Id')
required_column_order.insert(0, 'Id')

# Reorder your DataFrame columns
submission_df = submission_df[required_column_order]


In [ ]:
submission_df
submission_df.to_csv('/kaggle/working/submission_ordered_new.csv', index=False)


In [ ]:
submission_df

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Assume 'y_true' contains your true labels and 'y_pred' contains your model predictions
y_true = targets  # Example true labels
y_pred = outputs  # Example predictions

# Compute the confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
model_news= MultiLabelSciBERT(57)
model_path = '/kaggle/working/model1.pth'
loaded_st = torch.load(model_path)
model_news.load_state_dict(loaded_st)